## Integrantes del equipo

- Myroslava Sánchez Andrade A01730712
- José Antonio Bobadilla García A01734433
- Karen Rugerio Armenta A01733228
- Alejandro Castro Reus A01731065


### Cambios realizados para obtener un 90% de accuracy:

- Reducir el batch size a 17
- Utilizar el modelo GRU
- Utilizar un Dropout de .95
- Utilizar un EMBEDDING_SIZE de 200
- 4 capas profundas de 60 neuronas cada una
- 11 épocas
- Learning rate de 0.0002

In [1]:
#!pip install torch==1.12.1
#!pip install torch==1.8
#!pip install torch==1.13.0

!pip install torch==1.12.1  torchtext==0.13.1 torchdata==0.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 21.6 MB/s 
     |████████████████████████████████| 140 kB 72.4 MB/s 
     |████████████████████████████████| 127 kB 52.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import torch
print(torch. __version__)

1.12.1+cu113


In [4]:
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
from torch import nn
from torch.nn import functional as F
# Dataset and dataloader
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

In [5]:
torch.cuda.is_available()

True

In [6]:
# Use Gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
from torchtext.datasets import AG_NEWS

In [8]:
train_dataset,  test_dataset = AG_NEWS()

In [9]:
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [10]:
len(train_dataset), len(test_dataset)

(120000, 7600)

In [11]:
train_dataset[:10]

[(3,
  "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."),
 (3,
  'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'),
 (3,
  "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums."),
 (3,
  'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.'),
 (3,
  'Oil prices soar to all-time record, 

In [12]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
  for _, text in data:
    yield tokeniser(text)

In [13]:
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [14]:
len(vocab)

95811

In [15]:
for i in range(20):
  print(i, vocab.lookup_token(i))

0 <unk>
1 .
2 the
3 ,
4 to
5 a
6 of
7 in
8 and
9 s
10 on
11 for
12 #39
13 (
14 )
15 -
16 '
17 that
18 with
19 as


In [16]:
tokens = tokeniser('Welcome to TE3007B')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007b'] [3314, 4, 0]


In [17]:
NUM_TRAIN = int(len(train_dataset) * 0.9)
NUM_VAL =len(train_dataset) - NUM_TRAIN

In [18]:
NUM_VAL

12000

In [19]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [20]:
len(train_dataset), len(val_dataset), len(test_dataset)

(108000, 12000, 7600)

In [21]:
max_tokens = 50

In [22]:
def collate_batch(batch):
  y, x = list(zip(*batch))
  #create list with tokens
  x = [vocab(tokeniser(text)) for text in x]
  # padding or clipping
  x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]
  return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)-1

In [48]:
labels = ['World', 'Sports', 'Business', 'Sci/Tech']
# Reduce batch size to 17 

BATCH_SIZE = 17


In [49]:
# DataLoaders
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)
val_loader = DataLoader(val_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)
test_loader = DataLoader(test_dataset, 
                          batch_size=BATCH_SIZE,
                          collate_fn=collate_batch,
                          shuffle = True)

In [50]:
for i, (x, y) in enumerate(test_loader):
  print(i, x.shape, y.shape)

0 torch.Size([17, 50]) torch.Size([17])
1 torch.Size([17, 50]) torch.Size([17])
2 torch.Size([17, 50]) torch.Size([17])
3 torch.Size([17, 50]) torch.Size([17])
4 torch.Size([17, 50]) torch.Size([17])
5 torch.Size([17, 50]) torch.Size([17])
6 torch.Size([17, 50]) torch.Size([17])
7 torch.Size([17, 50]) torch.Size([17])
8 torch.Size([17, 50]) torch.Size([17])
9 torch.Size([17, 50]) torch.Size([17])
10 torch.Size([17, 50]) torch.Size([17])
11 torch.Size([17, 50]) torch.Size([17])
12 torch.Size([17, 50]) torch.Size([17])
13 torch.Size([17, 50]) torch.Size([17])
14 torch.Size([17, 50]) torch.Size([17])
15 torch.Size([17, 50]) torch.Size([17])
16 torch.Size([17, 50]) torch.Size([17])
17 torch.Size([17, 50]) torch.Size([17])
18 torch.Size([17, 50]) torch.Size([17])
19 torch.Size([17, 50]) torch.Size([17])
20 torch.Size([17, 50]) torch.Size([17])
21 torch.Size([17, 50]) torch.Size([17])
22 torch.Size([17, 50]) torch.Size([17])
23 torch.Size([17, 50]) torch.Size([17])
24 torch.Size([17, 50]) to

In [51]:
#let us build our RNN

In [52]:
# Changing number of Embedding size, neurons and layers.

EMBEDDING_SIZE = 200
NEURONS = 60 #hidden
LAYERS = 4
NUM_CLASSES = 4

In [61]:
class RNN_Model_1(nn.Module):
  def __init__(self, embed_size, hidden, layers, num_classes):
    super().__init__()
    self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                        embedding_dim=embed_size)
    
    # Using GRU model.
    self.rnn = nn.GRU(input_size=embed_size,
                      hidden_size = hidden,
                      num_layers = layers,
                      batch_first = True)


    self.fc = nn.Linear(in_features=hidden, out_features= num_classes)

    # using a dropout of .95
    self.dropout = nn.Dropout(0.95) # Adding dropout

  def forward(self, x):
    vector_embs = self.embedding_layer(x)
    y, h = self.rnn(vector_embs)
    return self.fc(y[:,-1])

In [62]:
rnn_model = RNN_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)

In [63]:
rnn_model

RNN_Model_1(
  (embedding_layer): Embedding(95811, 200)
  (rnn): GRU(200, 60, num_layers=4, batch_first=True)
  (fc): Linear(in_features=60, out_features=4, bias=True)
  (dropout): Dropout(p=0.95, inplace=False)
)

In [64]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype = torch.long)
            y = y.to(device=device, dtype = torch.long)
            scores = model(x)
            _, preds = scores.max(dim=1)
            num_correct += (preds == y).sum()
            num_total += preds.size(0)
        acc = float(num_correct)/num_total
        return acc

In [65]:
def train(model, optimiser, epochs=100):
  model = model.to(device=device)
  for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
      model.train()
      x = x.to(device=device, dtype=torch.long)
      y = y.to(device=device, dtype=torch.long)
      # run model
      scores = model(x)
      # compute cost
      cost = F.cross_entropy(input=scores, target=y)
      # reset gradient
      optimiser.zero_grad()
      # compute gradient
      cost.backward()
      # update parameter
      optimiser.step()
    acc = accuracy(model, val_loader)
    print(f'Epoch {epoch}, costo {cost.item():.4f}, val acc {acc:.4f}')


In [71]:

# Using only 11 epochs and a learning rate of 0.0002
epochs = 11
lr = 0.0002

rnn_model = RNN_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)
optimiser = torch.optim.Adam(rnn_model.parameters(), lr=lr)

In [72]:
train(rnn_model, optimiser, epochs)

Epoch 0, costo 0.0503, val acc 0.8711
Epoch 1, costo 0.0713, val acc 0.8871
Epoch 2, costo 0.4533, val acc 0.9032
Epoch 3, costo 0.3846, val acc 0.9049
Epoch 4, costo 0.0153, val acc 0.9063
Epoch 5, costo 0.0714, val acc 0.9032
Epoch 6, costo 0.1644, val acc 0.9042
Epoch 7, costo 0.0478, val acc 0.9022
Epoch 8, costo 0.0658, val acc 0.9009
Epoch 9, costo 0.0105, val acc 0.8998
Epoch 10, costo 0.0048, val acc 0.8994


In [73]:
print(f'{accuracy(rnn_model, test_loader):.4f}')

0.9050
